In [36]:
import pandas as pd

df = pd.read_csv('/media/r2-d2/E0C494CDC494A6F6/doutorado/rec_info/data/estupro_teste.csv', sep = ';', encoding = 'utf-8')



In [37]:
df

,HISTORICO,CONTEXTO
0,Comparece neste plantão os Policiais Milit...,COTIDIANO
1,Comparece a declarante informando que viveu co...,COTIDIANO
2,"Presente o declarante, informando qu...",COTIDIANO
3,Comparece nesta especializada a senhora Tatian...,COTIDIANO
4,Comparecem nesta delegacia os policiais milita...,COTIDIANO
...,...,...
2394,Comparece neste plantão os policiais militare...,UNICO
2395,"Presente a declarante informando que, na tarde...",UNICO
2396,Comparecem os GCMs Luciane e Detz apresentando...,UNICO
2397,Comparece nesta Unidade Policial o policial mi...,UNICO


In [38]:
# baixando as libs para pre processamento

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
from unidecode import unidecode


nltk.download('stopwords')

stopwords = set(stopwords.words('portuguese'))

stemmer = PorterStemmer()
#nltk.download("wordnet")
#from nltk.stem import WordNetLemmatizer
#lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /home/r2-d2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [39]:
# pre processamento

    # Remover caracteres especiais e números
    #text = re.sub('[^A-Za-z\s\']+', '', text)

def preprocess_text(text):

    padrao_arroba = re.compile(r'@\S+')
      # Remover as strings com "@" no início da string
    text = re.sub(padrao_arroba, '', text)

        #remover sites
    #pattern_url = re.compile(r'https?:\/\/.*?[\s+]')

    text = re.sub(r'http\S+', '', text)
    

    # Converter tudo para letras minúsculas
    text = text.lower()

    
    # Remover as tags HTML
    text = re.sub('<[^<]+?>', '', text)    

    text = unidecode(text)
    
    
    # Tokenizar o texto em palavras
    words = nltk.word_tokenize(text)

    # remover palavras que so possua caractere especial
    words = [word for word in words if not re.match('^[^A-Za-z0-9]+$', word)]

    # remover palavras que tenha tamanho 1
    words = [word for word in words if len(word) > 2]

    
    # Remover as stopwords
    words = [word for word in words if word not in stopwords]
    
    # Aplicar o stemming
    words = [stemmer.stem(word) for word in words]
    #words = [lemmatizer.lemmatize(word) for word in words]

    # Juntar as palavras em uma única string
    text = ' '.join(words)
  
    return text

In [40]:
# rodar o pre processamento

def preprocess_collection(docs):

    preprocessed_documents = []

    #for document in docs:
    for index, row in docs.iterrows():
        tema = row['CONTEXTO']
        #nome_arquivo = document['nome_arquivo']
        conteudo = row['HISTORICO']
        
        preprocessed_conteudo = preprocess_text(conteudo)
        
        preprocessed_document = {
            'tema': tema,
        #    'nome_arquivo': nome_arquivo,
            'conteudo': preprocessed_conteudo
        }
        
        preprocessed_documents.append(preprocessed_document)
        
    return preprocessed_documents

In [41]:
df.head(4)

,HISTORICO,CONTEXTO
0,Comparece neste plantão os Policiais Milit...,COTIDIANO
1,Comparece a declarante informando que viveu co...,COTIDIANO
2,"Presente o declarante, informando qu...",COTIDIANO
3,Comparece nesta especializada a senhora Tatian...,COTIDIANO


In [42]:
preprocessed_documents = preprocess_collection(df)

In [43]:
len(preprocessed_documents)

2399

In [44]:
frases = [o['conteudo'] for o in preprocessed_documents]

In [45]:
rotulos = []
for o in preprocessed_documents:
    rotulos.append(o['tema'])

In [46]:
len(rotulos)

2399

In [47]:
def criar_indice(preprocessed_documents):
    indice = {}
    for doc in preprocessed_documents:
        conteudo = doc['conteudo']
        if conteudo not in indice:
            indice[conteudo] = doc['tema']
    return indice




In [48]:
# Criar o índice uma vez antes de chamar a função recupera_rotulo
indice = criar_indice(preprocessed_documents)


In [49]:
def recupera_rotulo(texto):
    return indice.get(texto)

In [50]:
preprocessed_documents[101]['conteudo']

'comparec especializada declarant informando avo paterna vitima taina filho separado genitora vitima tre ano fruto dest relacionamento possuem doi filho menor vitima morava genitora ate me julho 2020 razao constant briga genitora atual companheiro gean declarant passou cuidar neto fruto relacionamento alex filho maraisa pai vitima neta taina contou prima yasmim ano mae trabalhar ficava sozinha padrasto mandava irmao buscar seda produto usado enrolar cigarro maconha segundo declarant vitima ficava sozinha autor entao autor mandava vitima tirar roupa subia cima fazia coisa declarant pergunt crianca autor colocava negocio nela mesma diss sim doia chorava mandava autor parar autor chamava putinha sempr genitora vitima trabalhar autor aproveitava abusava mesma afirma perguntado menina quanta veze havia ocorrido respondeu varia veze'

In [51]:
from rank_bm25 import BM25Okapi

tokenized_corpus = [doc.split(" ") for doc in frases]

bm25 = BM25Okapi(tokenized_corpus)

In [21]:
#dataset de teste

import pandas as pd

df_test = pd.read_csv('Corona_NLP_test.csv', encoding = "ISO-8859-1")
df_test_new = df_test[['OriginalTweet', 'Sentiment']]

In [22]:
df_test_sorted = df_test_new.sort_values('Sentiment')

In [23]:
data_test = preprocess_collection(df_test_sorted)

In [32]:
preprocessed_documents[19000]

{'tema': 'EVENTUAL',
 'conteudo': 'comparec vitima nesta unidad polici informando investigado vizinho resid terreno casa frent relata nao nenhum tipo contato portanto nao sabe informar qualificaco conta data fato havia pedido ajuda investigado levass cesta basica dentro casa vitima vez havia sido deixada portao vitima nao pode carregar peso possuir doenca afeta coluna nest momento dentro casa vitima investigado violent sexualment mantendo conjuncao carnal nest ato expedido iml nada'}

In [53]:
# apenas para teste
tokenized_query = preprocessed_documents[100]['conteudo'].split(" ")
results = bm25.get_top_n(tokenized_query, frases, n=len(preprocessed_documents))

for result in results:
    print(recupera_rotulo(result) + " " + result)

COTIDIANO compareceu nesta especializada representant vitima informando genitora desta filha hoje ano representant autor pai vitima casado ano hoje sao separado ano dessa relacao filho maior idad vitima filha passou ter contato present autor ultimo sei mese apo revisao guarda solicitada autor decisao anexa data fato filha residencia autor desd sabado manha hora dormir autor coloc vitima dormir sala colchao irmao dormiu quarto autor diss ainda esposa dormir sala junto camili condicionado senao nao conseguir dormir filha relat assistiram film dormindo volta 03:00 hora manha acord pai tocando part intima bem corpo seio sentiu pai chegou forcar dedo genitalia sentiu dor tentando afastar pai continuava ato corpo passando mao genitalia ate parou representant tambem relat logo manha autor levar filha casa durant trajeto tentou novament passar mao corpo bem forcou vitima tocar orgao genit ond vitima recus ainda autor sobr pressao deu importancia 100,00 cem reai vitima nao contass nada ninguem 

In [54]:
array_temas = ['COTIDIANO',
'EVENTUAL',
'UNICO']
temas = {}

for d in array_temas:
    temas[d] = {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}

print(temas)    

{'COTIDIANO': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'EVENTUAL': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'UNICO': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}}


In [55]:
def calcula_map(ranking, tema):
    acertos = 0
    maps = []
    for i, doc in enumerate(ranking):
        tema_achado = recupera_rotulo(doc)
        if tema == tema_achado:
            acertos +=1
            maps.append(acertos/(i+1))
    if len(maps) > 0:
        return mean(maps)
    else:
        print("não achou nada")
        return 0.0

In [58]:
from statistics import mean

from IPython.display import clear_output

# consultando
p10 = []
p20 = []
p50 = []
p100 = []
map_f = []
tema_atual = ""

for c, doc_consulta in enumerate(preprocessed_documents):
    clear_output(wait=True)
    print(c)
    #ranking = similaridade(doc_consulta['conteudo'], preprocessed_documents, idfs)
    tokenized_query = doc_consulta['conteudo'].split(" ")
    ranking = bm25.get_top_n(tokenized_query, frases, n=len(preprocessed_documents))
    acertos = 0
    
    map_f.append(calcula_map(ranking, doc_consulta['tema']))
    
    for i, doc in enumerate(ranking):
        #print(f"Documento {doc[0]} - Target {doc[1]} -->  {doc[2]} - Similaridade: {doc[3]}")
        #if doc[1] == doc_consulta['tema']:
        #print(doc_consulta['tema'])
        tema_achado = recupera_rotulo(doc)
        if doc_consulta['tema'] == tema_achado:
            #print(f"{doc_consulta['tema']} e {doc}")
            acertos+=1
        if i+1 == 10:
            p10.append(acertos/10.0)            
        if i+1 == 20:
            p20.append(acertos/20.0)
        if i+1 == 50:
            p50.append(acertos/50.0)
        if i+1 == 100:
            p100.append(acertos/100.0) 
        #map.append(calcula_map(ranking, doc_consulta['tema']))
    if c > 0:
        if tema_atual != doc_consulta['tema'] or c == len(preprocessed_documents) - 1:
            temas[tema_atual]['p10'] = mean(p10)
            temas[tema_atual]['p20'] = mean(p20)
            temas[tema_atual]['p50'] = mean(p50)
            temas[tema_atual]['p100'] = mean(p100)
            temas[tema_atual]['map'] = mean(map_f)
            p10 = []
            p20 = []
            p50 = []
            p100 = []
            #map = []

    tema_atual = doc_consulta['tema']
 


2398


In [59]:
temas

{'COTIDIANO': {'p10': 0.5691635455680399,
  'p20': 0.5265293383270911,
  'p50': 0.4968789013732834,
  'p100': 0.4740074906367041,
  'map': 0.38743800237289977},
 'EVENTUAL': {'p10': 0.49774718397997497,
  'p20': 0.4556946182728411,
  'p50': 0.4284105131414268,
  'p100': 0.41428035043804756,
  'map': 0.38040189188540624},
 'UNICO': {'p10': 0.6148936170212767,
  'p20': 0.5822277847309136,
  'p50': 0.5573967459324155,
  'p100': 0.5334918648310388,
  'map': 0.3993200217327482}}

In [60]:
import json
with open('/media/r2-d2/E0C494CDC494A6F6/doutorado/rec_info/results/est/est_bm25_menor.json', 'w') as f:
    json.dump(temas, f)

In [61]:
import pandas as pd

df = pd.DataFrame(temas)

In [62]:
df =df.transpose()

In [63]:
df

,p10,p20,p50,p100,map
COTIDIANO,0.569164,0.526529,0.496879,0.474007,0.387438
EVENTUAL,0.497747,0.455695,0.428411,0.414280,0.380402
UNICO,0.614894,0.582228,0.557397,0.533492,0.399320


In [64]:
df.describe()

,p10,p20,p50,p100,map
count,3.000000,3.000000,3.000000,3.000000,3.000000
mean,0.560601,0.521484,0.494229,0.473927,0.389053
std,0.059041,0.063417,0.064534,0.059606,0.009562
min,0.497747,0.455695,0.428411,0.414280,0.380402
25%,0.533455,0.491112,0.462645,0.444144,0.383920
50%,0.569164,0.526529,0.496879,0.474007,0.387438
75%,0.592029,0.554379,0.527138,0.503750,0.393379
max,0.614894,0.582228,0.557397,0.533492,0.399320
